# PyCity Schools - Homework 4 - Tom Callegari

### 3 Noticeable Trends

* The percentage of overall passing students summary table clearly defines Charter schools as having better performances (top 8 schools by % Overall Passing Rate) than District schools (bottom 7 schools by % Overall Passing Rate).
* Counter to what might be natural intuition, the more that is spent by a school per student does not guarantee success with the school spending summary showing that that schools who spent less per student have higher percentage overall passing rates than those who spent more.
* It appears that the larger a school is the less success it will have with overall passing rate percentages.


In [1]:
# Import libraries to be used
import pandas as pd
import numpy as np

In [2]:
# Read in schools_complete.csv and students.complet.csv
schools_raw = pd.read_csv('schools_complete.csv')
students_raw = pd.read_csv('students_complete.csv')

In [3]:
# Combine the dataframes into one 
schools = pd.merge(schools_raw, students_raw, how='left', on=['school_name', 'school_name'])
schools.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


### District Summary

* Calculate the total number of schools
* Calculate the total number of students
* Calculate the total budget
* Calculate the average math score
* Calculate the average reading score
* Calculate the oeverall passing rate (overall average score), i.e. (avg. math score + avg. reading score) / 2
* Calculate the percentage of students with a passing math score (70 or greater)
* Calcualte the percentage of students with a passing reading score (70 or greater)
* Create a dataframe to hold the abover results
* Optional: Give the displayed data cleaner formatting

In [4]:
# Calculate summary table statistics
total_schools = schools['school_name'].nunique()
total_students = schools['Student ID'].nunique()
total_budget = schools_raw['budget'].sum()
mean_math_score = schools['math_score'].mean()
mean_reading_score = schools['reading_score'].mean()

# Calculate passing percentages (math, reading & overall)
schools['passed_math'] = pd.cut(schools['math_score'],
                                [0, 69, 100],
                               labels = ['0', '1'])

schools['passed_reading'] = pd.cut(schools['reading_score'],
                                 [0, 69, 100],
                                 labels = ['0', '1'])

math_results = schools['passed_math'].value_counts()
reading_results = schools['passed_reading'].value_counts()

math_passing_perc = (math_results[1] / (math_results[0] + math_results[1])) * 100
reading_passing_perc = (reading_results[1] / (reading_results[0] + reading_results[1])) * 100
overall_passing_perc = (math_passing_perc + reading_passing_perc) / 2

# Assemble the needed data into a dictionary with f strings to add ($, %) symbols
district_summary_dict = {
    'Total Schools': total_schools,
    'Total Students': total_students,
    'Total Budget': total_budget,
    'Average Math Score': mean_math_score,
    'Average Reading Score': mean_reading_score,
    '% Passing Math': math_passing_perc,
    '% Passing Reading': reading_passing_perc,
    '% Overall Passing Rate': overall_passing_perc
}

# Insert the summary dictionary into a dataframe named district_summary
district_summary = pd.DataFrame(district_summary_dict, index=['Results'])

# Give the displayed data cleaner formatting
district_summary = district_summary.round(2)
district_summary = district_summary.style.format({
    'Total Students': '{:,.0f}'.format,
    'Total Budget': '${:,.2f}'.format,
    '% Passing Math': '{}%',
    '% Passing Reading': '{}%',
    '% Overall Passing Rate': '{}%'
})

# Display the summary table
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Results,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.39%


### School Summary

* Create an overview table that summarizes key metrics about each school including:
    * School Name
    * School Type
    * Total Students
    * Total School Budget
    * Per Student Budget
    * Average Math Score
    * Average Reading Score
    * % Passing Math
    * % Passing Reading
    * Overall Passing Rate (Average of the above two)

* Create a dataframe to hold the above results

In [5]:
schools_summary = schools.groupby(['school_name','type']).agg({
    'Student ID': 'count', 
    'budget': 'mean', 
    'math_score': 'mean',
    'reading_score': 'mean'    
})
schools_summary = schools_summary.reset_index()

math_grouped = schools.groupby('school_name').agg({'passed_math': 'value_counts'})

math_grouped = math_grouped.rename(columns={'passed_math': '% Passing Math'})
math_grouped = math_grouped.groupby(level=0).apply(lambda x: 100 * x / int(x.sum()))
math_grouped = math_grouped.reset_index()
math_grouped = math_grouped.iloc[::2]
math_grouped = math_grouped.drop('passed_math', axis=1)

reading_grouped = schools.groupby('school_name').agg({'passed_reading': 'value_counts'})

reading_grouped = reading_grouped.rename(columns={'passed_reading': '% Passing Reading'})
reading_grouped = reading_grouped.groupby(level=0).apply(lambda x: 100 * x / int(x.sum()))
reading_grouped = reading_grouped.reset_index()
reading_grouped = reading_grouped.iloc[::2]
reading_grouped = reading_grouped.drop('passed_reading', axis=1)

schools_summary = pd.merge(schools_summary, math_grouped, how='left', on='school_name')
schools_summary = pd.merge(schools_summary, reading_grouped, how='left', on='school_name')

schools_summary['% Overall Passing Rate'] = ((schools_summary['% Passing Math'] + schools_summary['% Passing Reading']) / 2) / 100
schools_summary['% Passing Math'] = schools_summary['% Passing Math'] / 100
schools_summary['% Passing Reading'] = schools_summary['% Passing Reading'] / 100

schools_summary = schools_summary.rename(
    columns={'school_name': 'School Name', 
             'type': 'School Type', 
             'Student ID': 'Total Students',
             'budget': 'Total School Budget', 
             'math_score': 'Average Math Score', 
             'reading_score': 'Average Reading Score'})

schools_summary['Per Student Budget'] = schools_summary['Total School Budget'] / schools_summary['Total Students']

schools_summary = schools_summary[[
    'School Name',
    'School Type',
    'Total Students',
    'Total School Budget',
    'Per Student Budget',
    'Average Math Score',
    'Average Reading Score',
    '% Passing Math',
    '% Passing Reading',
    '% Overall Passing Rate']]

schools_summary = schools_summary.set_index('School Name')

top_schools_summary = schools_summary.sort_values('% Overall Passing Rate', ascending=False).head(5)

top_schools = top_schools_summary.style.format({
    'Total School Budget': '${:,.2f}'.format,
    'Per Student Budget': '${:,.2f}'.format,
    'Average Math Score': '{:.2f}'.format,
    'Average Reading Score': '{:.2f}'.format,
    '% Passing Math': '{:.2%}'.format,
    '% Passing Reading': '{:.2%}'.format,
    '% Overall Passing Rate': '{:.2%}'.format
})

bottom_schools_summary = schools_summary.sort_values('% Overall Passing Rate', ascending=True).head(5)

bottom_schools = bottom_schools_summary.style.format({
    'Total School Budget': '${:,.2f}'.format,
    'Per Student Budget': '${:,.2f}'.format,
    'Average Math Score': '{:.2f}'.format,
    'Average Reading Score': '{:.2f}'.format,
    '% Passing Math': '{:.2%}'.format,
    '% Passing Reading': '{:.2%}'.format,
    '% Overall Passing Rate': '{:.2%}'.format
})

### Top Performing Schools (By Overall Passing Rate)

* Sort and display the top five schools in overall passing rate

In [6]:
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,95.29%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,95.27%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,95.20%


### Bottom Performing Schools (By Overall Passing Rate)

* Sort and display the five worst-performing schools

In [7]:
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%


### Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school
    * Create a pandas series for each grade.  Hint: Use a conditional statement
    * Group each series by school
    * Combine the series into a dataframe
    * Optional: Give the displayed data cleaner formatting

In [8]:
ninth_grade_math = schools[schools['grade'] == '9th'].groupby('school_name').agg({'math_score': 'mean'})
tenth_grade_math = schools[schools['grade'] == '10th'].groupby('school_name').agg({'math_score': 'mean'})
eleventh_grade_math = schools[schools['grade'] == '11th'].groupby('school_name').agg({'math_score': 'mean'})
twelfth_grade_math = schools[schools['grade'] == '12th'].groupby('school_name').agg({'math_score': 'mean'})

nine_ten_math = pd.merge(ninth_grade_math, tenth_grade_math, how='left', on='school_name')
nine_ten_math = nine_ten_math.rename(columns={'math_score_x': '9th Grade', 'math_score_y': '10th Grade'})

eleven_twelve_math = pd.merge(eleventh_grade_math, twelfth_grade_math, how='left', on='school_name')
eleven_twelve_math = eleven_twelve_math.rename(columns={'math_score_x': '11th Grade', 'math_score_y': '12th Grade'})

math_scores = pd.merge(nine_ten_math, eleven_twelve_math, how='left', on='school_name')

math_scores = math_scores.reset_index()

math_scores = math_scores.rename(columns={'school_name': 'School Name'})

math_scores = math_scores.set_index('School Name')

math_scores = math_scores.style.format({
    '9th Grade': '{:.2f}'.format,
    '10th Grade': '{:.2f}'.format,
    '11th Grade': '{:.2f}'.format,
    '12th Grade': '{:.2f}'.format
})

math_scores

,9th Grade,10th Grade,11th Grade,12th Grade
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


### Reading Score by Grade

* Perform the same operation as above for reading scores

In [9]:
ninth_grade_reading = schools[schools['grade'] == '9th'].groupby('school_name').agg({'reading_score': 'mean'})
tenth_grade_reading = schools[schools['grade'] == '10th'].groupby('school_name').agg({'reading_score': 'mean'})
eleventh_grade_reading = schools[schools['grade'] == '11th'].groupby('school_name').agg({'reading_score': 'mean'})
twelfth_grade_reading = schools[schools['grade'] == '12th'].groupby('school_name').agg({'reading_score': 'mean'})

nine_ten_reading = pd.merge(ninth_grade_reading, tenth_grade_reading, how='left', on='school_name')
nine_ten_reading = nine_ten_reading.rename(columns={'reading_score_x': '9th Grade', 'reading_score_y': '10th Grade'})

eleven_twelve_reading = pd.merge(eleventh_grade_reading, twelfth_grade_reading, how='left', on='school_name')
eleven_twelve_reading = eleven_twelve_reading.rename(columns={'reading_score_x': '11th Grade', 'reading_score_y': '12th Grade'})

reading_scores = pd.merge(nine_ten_reading, eleven_twelve_reading, how='left', on='school_name')

reading_scores = reading_scores.reset_index()

reading_scores = reading_scores.rename(columns={'school_name': 'School Name'})

reading_scores = reading_scores.set_index('School Name')

reading_scores = reading_scores.style.format({
    '9th Grade': '{:.2f}'.format,
    '10th Grade': '{:.2f}'.format,
    '11th Grade': '{:.2f}'.format,
    '12th Grade': '{:.2f}'.format
})

reading_scores

,9th Grade,10th Grade,11th Grade,12th Grade
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


### Scores by School Spending

* Create a table that breaks down school performance based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
    * Average Math Score
    * Average Reading Score
    * % Passing Math
    * % Passing Reading
    * Overall Passing Rate (Average of the above two)

In [10]:
schools_summary['Spending Ranges'] = pd.cut(
    schools_summary['Per Student Budget'], 
    bins=[0, 585, 615, 645, 675],
    labels=["<$585", "$585-615", "$615-645", "$645-675"])
schools_summary.head()

spending = schools_summary.groupby('Spending Ranges').agg({
    'Average Math Score': 'mean',
    'Average Reading Score': 'mean',
    '% Passing Math': 'mean',
    '% Passing Reading': 'mean',
})

spending = spending.reset_index()

spending['% Overall Passing Rate'] = ((spending['% Passing Math'] + spending['% Passing Reading']) /2)

spending = spending.set_index('Spending Ranges')

spending = spending.style.format({
    'Average Math Score': '{:.2f}'.format,
    'Average Reading Score': '{:.2f}'.format,
    '% Passing Math': '{:.2%}'.format,
    '% Passing Reading': '{:.2%}'.format,
    '% Overall Passing Rate': '{:.2%}'.format
})

spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges,,,,,
<$585,83.46,83.93,93.46%,96.61%,95.04%
$585-615,83.60,83.89,94.23%,95.90%,95.07%
$615-645,79.08,81.89,75.67%,86.11%,80.89%
$645-675,77.00,81.03,66.16%,81.13%,73.65%


### Scores by School Size

* Peform the same operations as above, based on school size

In [11]:
schools_summary['Size Ranges'] = pd.cut(
    schools_summary['Total Students'], 
    bins=[0, 1000, 2000, 5000], 
    labels=["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"])

sizes = schools_summary.groupby('Size Ranges').agg({
    'Average Math Score': 'mean',
    'Average Reading Score': 'mean',
    '% Passing Math': 'mean',
    '% Passing Reading': 'mean'
})

sizes = sizes.reset_index()

sizes['% Overall Passing Rate'] = ((sizes['% Passing Math'] + sizes['% Passing Reading']) / 2)

sizes = sizes.set_index('Size Ranges')

sizes = sizes.style.format({
    'Average Math Score': '{:.2f}'.format,
    'Average Reading Score': '{:.2f}'.format,
    '% Passing Math': '{:.2%}'.format,
    '% Passing Reading': '{:.2%}'.format,
    '% Overall Passing Rate': '{:.2%}'.format
})

sizes

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Size Ranges,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,94.82%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,95.20%
Large (2000-5000),77.75,81.34,69.96%,82.77%,76.36%


### Scores by School Type

* Perform the same operations as above, based on school type

In [12]:
types = schools_summary.groupby('School Type').agg({
    'Average Math Score': 'mean',
    'Average Reading Score': 'mean',
    '% Passing Math': 'mean',
    '% Passing Reading': 'mean'
})

types = types.reset_index()

types['% Overall Passing Rate'] = ((types['% Passing Math'] + types['% Passing Reading']) /2)

types = types.set_index('School Type')

types = types.style.format({
    'Average Math Score': '{:.2f}'.format,
    'Average Reading Score': '{:.2f}'.format,
    '% Passing Math': '{:.2%}'.format,
    '% Passing Reading': '{:.2%}'.format,
    '% Overall Passing Rate': '{:.2%}'.format
})

types

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,95.10%
District,76.96,80.97,66.55%,80.80%,73.67%
